<a href="https://colab.research.google.com/github/Krishnabhuvan/Indian-Cities-Housing-property-Price/blob/main/Indian_Cities_Housing_property_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/Makaan_Properties_Buy.csv',encoding = "latin-1", delimiter=',', low_memory=False)

In [ ]:
X = df[["Property_type","Property_status","Price_per_unit_area","Posted_On","Property_building_status",
       "City_name","No_of_BHK","Size","is_furnished","Price"]]

y = X[["Price"]]

In [ ]:
X.sample(10)

In [ ]:
X.shape

In [ ]:
X.isnull().sum()

In [ ]:
X.dropna(inplace=True)

In [ ]:
X.info()

MAPPING-VALUES

In [ ]:
df["Property_building_status"].value_counts()

In [ ]:
dict = {
    "UNVERIFIED" : 0,
    "ACTIVE":1,
    "INACTIVE":2
}
X['Property_building_status'] = df['Property_building_status'].map(dict)

In [ ]:
df['Property_type'].value_counts()

In [ ]:
dict = {
    "Apartment":0,
    "Independent Floor":1,
    "Independent House":2,
    "Penthouse":3,
    "Studio Apartment":4,
}
X['Property_type'] = df['Property_type'].map(dict)

In [ ]:
df["Property_status"].value_counts()


In [ ]:
dict = {
    "Ready to move" : 0,
    "Under Construction" : 1
}
X['Property_status'] = df['Property_status'].map(dict)

In [ ]:
df["is_furnished"].value_counts()

In [ ]:
dict = {
    "Unfurnished":0,
    "Semi-Furnished":1,
    "Furnished":2
}
X['is_furnished'] = df['is_furnished'].map(dict)


In [ ]:
df["City_name"].value_counts()

In [ ]:
dict = {
    "Mumbai" : 0,
    "Chennai" : 1,
    "Hyderabad" : 2,
    "Bangalore" : 3,
    "Lucknow" : 4,
    "Delhi" : 5,
    "Kolkata" : 6,
    "Ahmedabad" : 7
}

X['City_name'] = df['City_name'].map(dict)

Splitting

In [ ]:
X["Posted_days_ago"] = X["Posted_On"].str.split(" ").str[0].str.replace("a","1").str.replace("an","3").str.replace("1n","1")
X = X.drop(columns=["Posted_On"],axis = 1)

In [ ]:
X["Size"] = X["Size"].str.split(" ").str[0]

In [ ]:
X["No_of_BHK"] = X["No_of_BHK"].str.split(" ").str[0]

Feature scaling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X["Property_type"] = X["Property_type"].fillna(0).astype(int)

In [ ]:
X.head()

data cleaning

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
#data cleaning - both columns
def clean_value(value):
  return value.replace(',','').replace(' ','')

columns_to_clean = ["Price_per_unit_area","Size"]
for col in columns_to_clean:
  X[col] = X[col].apply(clean_value)


Data preprocessing

In [ ]:
#Convert text values into real numeric values
X["Price_per_unit_area"] = X["Price_per_unit_area"].astype(float)
X["No_of_BHK"]=X["No_of_BHK"].astype(int)
X["Size"]=X["Size"].astype(int)
# Drop rows with missing values in 'Posted_days_ago' before converting to int
X.dropna(subset=['Posted_days_ago'], inplace=True)
X["Posted_days_ago"] = X["Posted_days_ago"].astype(int)

In [ ]:
y = X[["Price"]]
y.head(2)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
def clean_value(value):
    return value.replace(',', '').replace(' ', '')

# Apply cleaning function to the specified columns
columns_to_clean = ["Price"]
for col in columns_to_clean:
    y[col] = y[col].apply(clean_value)


y["Price"] = y["Price"].astype(float)

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.isnull().sum()

In [ ]:
X = X.drop(columns=["Price"])

In [ ]:
X.head()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled


In [ ]:
y_train.head(1)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

In [ ]:
clf = Sequential()

clf.add(Dense(units=512, kernel_initializer="he_normal", activation="relu", input_dim=9))
clf.add(Dropout(0.2))

clf.add(Dense(units=412, kernel_initializer="he_normal", activation="relu"))
clf.add(Dropout(0.3))

clf.add(Dense(units=350, kernel_initializer="he_normal", activation="relu"))
clf.add(Dropout(0.3))

clf.add(Dense(units=250, kernel_initializer="he_normal", activation="relu"))
clf.add(Dropout(0.2))

clf.add(Dense(units=100, kernel_initializer="he_normal", activation="relu"))
clf.add(Dropout(0.2))

clf.add(Dense(units=1, activation="linear"))


clf.compile(optimizer="adam", loss="mean_squared_logarithmic_error", metrics=[tf.keras.metrics.RootMeanSquaredError()])
clf.summary()

history = clf.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), batch_size=100, epochs=70)


In [ ]:
from sklearn.metrics import r2_score

y_pred = clf.predict(X_test_scaled)

r2_score(y_test, y_pred)

In [ ]:
plt.plot(history.history["root_mean_squared_error"])
plt.plot(history.history["val_root_mean_squared_error"])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
import pickle
from keras.models import load_model

with open('buy_model.pickle', 'wb') as f:
    pickle.dump(clf, f)

clf.save('buy_model.h5')

In [ ]:
import pandas as pd

In [ ]:
pf = pd.read_csv(r'/content/Makaan_Properties_Buy.csv', encoding='latin-1')

In [ ]:
pip install ydata-profiling


In [ ]:
from ydata_profiling import ProfileReport
prof = ProfileReport(pf)
prof.to_file('output.html')
